# ANN vs SNN Equivalence Demo

This notebook demonstrates that a Spiking Neural Network (SNN) using Sigma-Delta neurons 
produces the same output as a traditional Artificial Neural Network (ANN) with ReLU activation,
when both networks have identical weights.

## Architecture

```
Input (3) → [Dense W1] → Hidden (4, ReLU) → [Dense W2] → Output (2, ReLU)
```

## Key Insight

Sigma-Delta neurons work by:
1. **Delta encoder**: Encodes changes in input as sparse spikes
2. **Sigma decoder**: Accumulates spikes to reconstruct continuous values
3. **Activation**: Applies ReLU (same as ANN)

For constant input, Delta sends the value once, then zeros. Sigma holds it.


## 1. Setup and Imports


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Lava imports
from lava.proc.io.source import RingBuffer as InputBuffer
from lava.proc.dense.process import Dense
from lava.proc.sdn.process import Sigma, Delta, SigmaDelta, ActivationMode
from lava.proc.monitor.process import Monitor
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi2SimCfg

np.random.seed(42)  # For reproducibility
print("Imports successful!")


## 2. Define Network Architecture and Random Weights


In [ ]:
# Network dimensions
INPUT_SIZE = 3
HIDDEN_SIZE = 4
OUTPUT_SIZE = 2

# Generate random weights (same for both ANN and SNN)
W1 = np.random.randn(HIDDEN_SIZE, INPUT_SIZE) * 0.5  # Shape: (4, 3)
W2 = np.random.randn(OUTPUT_SIZE, HIDDEN_SIZE) * 0.5  # Shape: (2, 4)

print(f"W1 shape: {W1.shape}")
print(f"W1:\n{W1}\n")
print(f"W2 shape: {W2.shape}")
print(f"W2:\n{W2}")


## 3. Traditional ANN (NumPy)

A simple feedforward network with ReLU activations.


In [ ]:
def relu(x):
    """ReLU activation function"""
    return np.maximum(x, 0)


def ann_forward(x, w1, w2):
    """
    Forward pass through 2-layer ANN.
    
    Args:
        x: Input vector (INPUT_SIZE,)
        w1: First layer weights (HIDDEN_SIZE, INPUT_SIZE)
        w2: Second layer weights (OUTPUT_SIZE, HIDDEN_SIZE)
    
    Returns:
        Output vector (OUTPUT_SIZE,)
    """
    # Layer 1: Input -> Hidden
    h = relu(w1 @ x)
    
    # Layer 2: Hidden -> Output
    y = relu(w2 @ h)
    
    return y, h  # Return hidden activations too for debugging


# Test input - positive values work better for ReLU demo
x_input = np.array([1.0, 0.8, 0.5])

# Run ANN
ann_output, ann_hidden = ann_forward(x_input, W1, W2)

print(f"Input: {x_input}")
print(f"Hidden layer (after ReLU): {ann_hidden}")
print(f"ANN Output: {ann_output}")


## 4. Spiking Neural Network (Lava with Sigma-Delta neurons)

### Architecture for SNN:
```
Input → Delta(encode) → Dense(W1) → SigmaDelta(hidden,ReLU) → Dense(W2) → SigmaDelta(output,ReLU)
```

The Delta encoder at the input:
- At t=0: sends the input value (change from 0)
- At t>0: sends 0 (no change in constant input)

This allows Sigma in SigmaDelta to accumulate once and hold the value.


In [ ]:
# SNN Parameters
T_SIM = 30       # Number of timesteps to run
VTH = 0.01       # Sigma-Delta threshold (smaller = more accurate)

# Prepare input data: constant input repeated over time
# Shape: (INPUT_SIZE, T_SIM)
inp_data = np.tile(x_input.reshape(-1, 1), (1, T_SIM))

print(f"Input data shape: {inp_data.shape}")
print(f"Input is constant: {x_input} repeated for {T_SIM} timesteps")


In [ ]:
# Build SNN
# ---------

# Input source (sends input values)
source = InputBuffer(data=inp_data)

# Identity projection from source to delta encoder
inp_proj = Dense(weights=np.eye(INPUT_SIZE), num_message_bits=24)

# Delta encoder for input - encodes changes only
inp_delta = Delta(
    shape=(INPUT_SIZE,),
    vth=VTH,
    cum_error=True
)

# Layer 1: Input -> Hidden
dense1 = Dense(weights=W1, num_message_bits=24)

# Hidden layer neurons (Sigma-Delta with ReLU)
hidden_neurons = SigmaDelta(
    shape=(HIDDEN_SIZE,),
    vth=VTH,
    act_mode=ActivationMode.RELU,
    cum_error=True
)

# Layer 2: Hidden -> Output
dense2 = Dense(weights=W2, num_message_bits=24)

# Output layer neurons (Sigma-Delta with ReLU)
output_neurons = SigmaDelta(
    shape=(OUTPUT_SIZE,),
    vth=VTH,
    act_mode=ActivationMode.RELU,
    cum_error=True
)

# Connect the network
# Input -> Delta encoder
source.s_out.connect(inp_proj.s_in)
inp_proj.a_out.connect(inp_delta.a_in)

# Delta encoder -> Layer 1
inp_delta.s_out.connect(dense1.s_in)
dense1.a_out.connect(hidden_neurons.a_in)

# Layer 1 -> Layer 2
hidden_neurons.s_out.connect(dense2.s_in)
dense2.a_out.connect(output_neurons.a_in)

# Monitor to record activations over time
monitor_out = Monitor()
monitor_out.probe(output_neurons.act, T_SIM)

monitor_hidden = Monitor()
monitor_hidden.probe(hidden_neurons.act, T_SIM)

# Also monitor sigma (accumulated input)
monitor_sigma_hidden = Monitor()
monitor_sigma_hidden.probe(hidden_neurons.sigma, T_SIM)

print("SNN built successfully!")


In [ ]:
# Run SNN simulation
output_neurons.run(
    condition=RunSteps(num_steps=T_SIM),
    run_cfg=Loihi2SimCfg(select_tag='floating_pt')
)

# Get recorded data
out_data = monitor_out.get_data()
hidden_data = monitor_hidden.get_data()
sigma_data = monitor_sigma_hidden.get_data()

# Stop the runtime
output_neurons.stop()

# Extract activation traces
out_proc_key = list(out_data.keys())[0]
hidden_proc_key = list(hidden_data.keys())[0]
sigma_proc_key = list(sigma_data.keys())[0]

snn_output_trace = out_data[out_proc_key]['act']
snn_hidden_trace = hidden_data[hidden_proc_key]['act']
snn_sigma_trace = sigma_data[sigma_proc_key]['sigma']

print(f"SNN output trace shape: {snn_output_trace.shape}")
print(f"SNN hidden trace shape: {snn_hidden_trace.shape}")


## 5. Compare ANN and SNN outputs


## 6. Visualization

Let's see how the SNN activations converge to the ANN values over time.


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

time_steps = np.arange(T_SIM)

# Plot Hidden layer activations
ax = axes[0, 0]
for i in range(HIDDEN_SIZE):
    ax.plot(time_steps, snn_hidden_trace[:, i], label=f'SNN Hidden[{i}]', linewidth=2)
    ax.axhline(y=ann_hidden[i], color=f'C{i}', linestyle='--', alpha=0.7)
ax.set_xlabel('Time step')
ax.set_ylabel('Activation')
ax.set_title('Hidden Layer: SNN (solid) vs ANN (dashed)')
ax.legend(loc='upper right', fontsize=8)
ax.grid(True, alpha=0.3)

# Plot Output layer activations
ax = axes[0, 1]
for i in range(OUTPUT_SIZE):
    ax.plot(time_steps, snn_output_trace[:, i], label=f'SNN Output[{i}]', linewidth=2)
    ax.axhline(y=ann_output[i], color=f'C{i}', linestyle='--', alpha=0.7,
               label=f'ANN Output[{i}] = {ann_output[i]:.4f}')
ax.set_xlabel('Time step')
ax.set_ylabel('Activation')
ax.set_title('Output Layer: SNN (solid) vs ANN (dashed)')
ax.legend(loc='upper right', fontsize=8)
ax.grid(True, alpha=0.3)

# Plot Sigma (accumulated input) for hidden layer
ax = axes[1, 0]
for i in range(HIDDEN_SIZE):
    ax.plot(time_steps, snn_sigma_trace[:, i], label=f'Sigma[{i}]', linewidth=2)
    # Expected sigma = W1 @ x_input (before ReLU)
    expected_sigma = (W1 @ x_input)[i]
    ax.axhline(y=expected_sigma, color=f'C{i}', linestyle='--', alpha=0.7)
ax.set_xlabel('Time step')
ax.set_ylabel('Sigma (accumulated input)')
ax.set_title('Hidden Layer Sigma: SNN (solid) vs Expected W1@x (dashed)')
ax.legend(loc='upper right', fontsize=8)
ax.grid(True, alpha=0.3)

# Bar chart comparison
ax = axes[1, 1]
x_pos = np.arange(HIDDEN_SIZE)
width = 0.35

# Hidden layer bars
ax.bar(x_pos - width/2, ann_hidden, width, label='ANN Hidden', alpha=0.8)
ax.bar(x_pos + width/2, snn_hidden, width, label='SNN Hidden', alpha=0.8)

ax.set_xlabel('Neuron index')
ax.set_ylabel('Activation')
ax.set_title('Final Activations: ANN vs SNN')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('ann_snn_comparison.png', dpi=150)
plt.show()

print("\nPlot saved to: ann_snn_comparison.png")


## 7. Conclusion

This demonstration shows that:

1. **SNN with Sigma-Delta neurons is functionally equivalent to ANN with ReLU**
2. **Same weights produce same outputs** (within quantization tolerance defined by `vth`)
3. **Delta encoding** is key - it encodes only changes, allowing Sigma to hold values

### Key Architecture for Equivalence

```
ANN:  Input → Dense(W) → ReLU → Output
SNN:  Input → Delta → Dense(W) → SigmaDelta(ReLU) → Output
```

### Practical Implications

- Take any pre-trained ANN weights
- Load them into an isomorphic SNN (using Sigma-Delta neurons)
- Get the same computational results
- Benefit from energy efficiency of sparse spike-based computation on Loihi2

### Parameters that affect accuracy

- `vth` (threshold): Lower = more accurate but more spikes
- `cum_error`: Cumulative error helps with precision
- `num_message_bits`: Higher = more precision in graded spikes


In [ ]:
# Get final SNN output (at last timestep, when it has converged)
snn_output = snn_output_trace[-1]
snn_hidden = snn_hidden_trace[-1]

print("=" * 60)
print("COMPARISON: ANN vs SNN")
print("=" * 60)

print(f"\nInput: {x_input}")

print(f"\n--- Hidden Layer ---")
print(f"ANN hidden: {ann_hidden}")
print(f"SNN hidden: {snn_hidden}")
print(f"Difference: {np.abs(ann_hidden - snn_hidden)}")
print(f"Max error:  {np.max(np.abs(ann_hidden - snn_hidden)):.6f}")

print(f"\n--- Output Layer ---")
print(f"ANN output: {ann_output}")
print(f"SNN output: {snn_output}")
print(f"Difference: {np.abs(ann_output - snn_output)}")
print(f"Max error:  {np.max(np.abs(ann_output - snn_output)):.6f}")

# Check if outputs match within tolerance
tolerance = VTH * 3  # Allow some tolerance based on threshold
hidden_match = np.allclose(ann_hidden, snn_hidden, atol=tolerance)
output_match = np.allclose(ann_output, snn_output, atol=tolerance)

print("\n" + "=" * 60)
if hidden_match and output_match:
    print(f"SUCCESS: ANN and SNN outputs match within tolerance ({tolerance:.4f})!")
else:
    print(f"Hidden match: {hidden_match}, Output match: {output_match}")
    print(f"Tolerance used: {tolerance:.4f}")
print("=" * 60)
